### Webscraper for extracting newly published names, strains and accession numbers from the weeekly IJSEM email (saved in html). Script then compares the IJSEM names to the NCBI names and generates a report used for taxonomy updates. 
### Uses Natural Language processing combined with REGEX to extract information from text
### Notes on selenium. There is a lot of bad advise in stack overflow that refers to old versions of selenium. Need to refer to the offocial documentation https://selenium-python.readthedocs.io/ If you run out of space in your linux directory the script will crash. There is a cache file written by selenium that may then need to be deleted to get the chrome driver working again. Follow the path in the error message and delete that cache directory. Selenium needs a webdriver installed, find details at https://sites.google.com/chromium.org/driver/downloads

### NOTE selenium cache files are quite large and need to be cleaned up periodically

In [ ]:
pip install "numpy<2.0" 

In [ ]:
pip install spacy
#pip install -U pip setuptools wheel
#pip install -U spacy
#pip install --upgrade spacy

### Install the model at the command lines
small model
python -m spacy download en_core_web_sm

medium model
python -m spacy download en_core_web_md

large model
python -m spacy download en_core_web_lg

### create training set https://spacy.io/usage/training

This annotator works better https://agateteam.org/spacynerannotate/ combined with the DocBin code to create the .spacy file. 

https://spacy.io/usage/training#quickstart

python -m spacy init fill-config ./base_config.cfg ./config.cfg

This NER annotator worked better than the one above. Used the DocBin technique to convert to .spacy file. Spacy training command with config.cfg then worked.

Config.cfg needs to be created for the spacy command line training. 

Follow the instructions in the quickstart for the base_config.cfg file. Selected OS and clicked NER

https://spacy.io/usage/training#quickstart

Run the following at command line to create the config.cfg. 

python -m spacy init fill-config ./base_config.cfg ./config.cfg

Once the config file was done and spacy file was created run the following at command line to train the model

To debug the data:
python -m spacy debug data config.cfg --paths.train ./train.spacy --paths.dev ./train.spacy

To debug the config file file:
python -m spacy train config.cfg --output ./output

or

python -m spacy train config.cfg 

train the model at command line
python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy

creates two directories model-best and model-last. I used model-best 

nlp1 = spacy.load(r"./output/model-best")


Notes
python -m spacy init fill-config ./base_config.cfg ./config.cfg

python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy

https://spacy.io/usage/training#basics

### Train spacy if not already done. Using output https://arunmozhi.in/ner-annotator/


In [ ]:
from spacy.tokens import DocBin
import pandas as pd
import json
import os

### train the model at command line
python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy 

### Main webscraper program start


In [8]:
import selenium
import tempfile
from selenium import webdriver
from selenium.webdriver.common.by import By
chrome_path = "/home/mcveigh/.cache/selenium/chrome/linux64/138.0.7204.183/chrome"
from selenium.webdriver.chrome.options import Options

import time
import IPython
import pandas as pd
import re
import os
import sys
import bs4
from bs4 import BeautifulSoup
import requests
import numpy as np
from datetime import datetime
from nameparser import HumanName
import spacy
#nlp = spacy.load("en_core_web_sm")
nlp = spacy.load("en_core_web_md")
from spacy.matcher import PhraseMatcher
from spacy import displacy

In [ ]:
#test
text2 = ("The type strain is 2205BS29-5T (=LMG 33062T =KACC 23240T), which is isolated from a marine sponge, P. elegans, "
"collected from Beomseom in Jeju-Island, Republic of Korea."

"The DNA G+C content of strain 2205BS29-5T is 67.8%. The GenBank accession numbers for the 16S rRNA gene and" 
"whole-genome sequences of strain 2205BS29-5T are OQ569368 and JAVAMQ000000000, respectively.")

In [ ]:
#test but need this later
import IPython
nlp1 = spacy.load(r"./output/model-best") #load the best model
doc = nlp1(text2)
from IPython.display import display
from spacy import displacy
from IPython.core.display import HTML, display
displacy.render(doc, style="ent")

### Find URLS from saved email in html - save as from outlook in htm format. URLs are extracted and saved as input for selenium

In [2]:
input = (r'IJSEMemail56.htm')
output = (r'NameCheckweek56a.xlsx')
alldescriptions = (r'all_descriptions56a')

In [ ]:
#Only need this if you suspect a problem with beautiful soup not imporing the URL list correctly. Otherwise skip this step
from bs4.diagnose import diagnose
with open (input, encoding = 'unicode_escape') as f:
    data = f.read()
diagnose(data)

#### alternative Beautifiul soup code for extracting URLS with autodetect encoding

In [3]:
#alternative Beautifiul soup with autodetect encoding
from charset_normalizer import from_path

# Auto-detect file encoding
result = from_path(input).best()
html = str(result)

# Parse with BeautifulSoup
soup = BeautifulSoup(html, "html.parser")
text = soup.get_text()

# Extract all http/https links
urls = re.findall(r"https?://\S+", text)
urls = [url.rstrip('.,);') for url in urls]

# Filter links as needed
filtered_urls = [
    url for url in urls
    if all(exclude not in url for exclude in ["TandC", "myaccount", "join-the-society"])
    # Remove "doi.org" from filter if you want article links
]

if not filtered_urls:
    raise ValueError("No usable URLs found!")

print(f"Found {len(filtered_urls)} URLs:")
print("\n".join(filtered_urls))


Found 10 URLs:
http://dx.doi.org/10.1099/ijsem.0.006793
http://dx.doi.org/10.1099/ijsem.0.006801
http://dx.doi.org/10.1099/ijsem.0.006792
http://dx.doi.org/10.1099/ijsem.0.006867
http://dx.doi.org/10.1099/ijsem.0.006861
http://dx.doi.org/10.1099/ijsem.0.006866
http://dx.doi.org/10.1099/ijsem.0.006853
http://dx.doi.org/10.1099/ijsem.0.006859
http://dx.doi.org/10.1099/ijsem.0.006856
http://dx.doi.org/10.1099/ijsem.0.006858


In [4]:
def find_strains(description):
    """Find strains with spacy"""
    #print('Entering strain subroutine')
    doc = nlp(description)
    nlp1 = spacy.load(r"./output/model-best") #load the best model
    phrase_matcher = PhraseMatcher(nlp.vocab)
    phrases = ['strain', 'Strain', 'strains', 'Strains']
    patterns = [nlp(description) for description in phrases]
    phrase_matcher.add('STRAIN', None, *patterns)
               
    for sent in doc.sents:    
        for match_id, start, end in phrase_matcher(nlp(sent.text)):
            if nlp.vocab.strings[match_id] in ["STRAIN"]:
                print(sent.text)
                doc = nlp1(sent.text)
                displacy.render(doc, style="ent")
                for entity in doc.ents:
                    if entity.text not in strains:
                        entity.text.encode('ascii', 'ignore').decode('unicode_escape')
                        strains.append(entity.text)
    return strains

def remove_non_ascii(text):
    """Remove non-ASCII characters"""
    return ''.join(char for char in text if ord(char) < 128)

### Main body - Selenium to extract data from html
### https://nameparser.readthedocs.io/en/latest/index.html to find the last name

In [9]:
from selenium.webdriver.common.by import By
import tempfile
pub_df = pd.DataFrame(columns=['PublishedName', 'Accessions', 'Strains', 'Authority', 'DOI', 'filtered_url'])
pd.set_option('display.max_columns', None)
combined_description = []

for filtered_url in filtered_urls:
    temp_profile = tempfile.mkdtemp()

    options = Options()
    options.binary_location = chrome_path
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=options)

    counter = 1
    strains = []
    accessions = []
    orgname = []
    doi = []
    author = []
    date = []
    year = []
    author_raw = []
    author = []
    name1 = []
    name2 = []
    author_count = []
    authority = []
    description = None
    description1 = []
    description2 = []
    snumber = []

    #Navigate to the webpage
    driver.get(filtered_url)

    #Allow time for dynamic content to load (you may need to use WebDriverWait for more robust waiting)
    time.sleep(3)
    
    html = driver.page_source
    
    for element in driver.find_elements(By.CLASS_NAME, "item-meta-data__item-title"):
        #print(element.text)
        title = element.text
        #print(title)
        
    for element in driver.find_elements(By.PARTIAL_LINK_TEXT, "doi.org"):
        doi = element.text
        #print(doi)
    
    #find publication year
    for element in driver.find_elements(By.XPATH, "//*[@id='bellowheadercontainer']/main/div[2]/div/ul/li[3]/span/span[2]"):
        date = element.text
        year = date[-4:]
        #print(year)
    
    #find authors
    for element in driver.find_elements(By.XPATH, "//*[@id='bellowheadercontainer']/main/div[2]/div/ul/li[1]/span"):
        author_raw = element.text
        #print(author_raw)
        author = re.sub(r"[\d+]+",'',author_raw)
        author = re.sub(r"†",'',author)
        author = re.sub(r" and ",',',author)
        author = re.sub(r",,",',',author)  
        author = author.split(',')
        author[:] = [item for item in author if item != '']
        #print(author)
        author_count = len(author)
        #print(author_count)
        if author_count == 1:
            name1 = (str(author[0])) 
            name1 = HumanName(name1)
            authority = name1.last + ' ' +str(year)
            #print(authority)
        elif author_count ==2:
            name1 = (str(author[0]))
            name1 = HumanName(name1)
            name2 = (str(author[1]))
            name2 = HumanName(name2)
            authority = name1.last + ' and ' + name2.last + ' ' +str(year)
            #print(authority)
        else:
            name1 = (str(author[0]))
            name1 = HumanName(name1)
            authority = name1.last + ' et al. ' +str(year)
            #print(authority)
    
    #extract data from each species description
    for element in driver.find_elements(By.CSS_SELECTOR, "div.tl-main-part.title"): #finds section headers
        #print(element.text)
        counter += 1
        description = element.text
        #print(description)
        if "Description of" in description:
            strains = []
            #print('found', description)  
            #snumber = 's' + str(counter - 4) + '/p[3]'
            snumber = 's' + str(counter - 4)
            #print('snumber is', snumber)
            for element in driver.find_elements(By.ID, snumber):
                description = element.text
                cleaned_text = remove_non_ascii(description)
                combined_description.append(cleaned_text)
                #print(cleaned_text)
                #print(description)
                        
                #find the organism names 
                orgname = []
                match = [r'(\S+\s+){2}(?=sp. nov)', r'(\S+\s+){2}(?=nom. nov.)', r'(\S+\s+){2}(?=SP. NOV.)', r'(\S+\s+){4}(?=subsp. nov.)']
                regex = re.compile(r'\b(' + '|'.join(match) + r')\b')
                if description is not None:
                    orgname = [m.group() for m in regex.finditer(description)]
                    #print('orgname', orgname)

                #find the accessions
                pattern = [r'[A-Z]{2}\d{6}', r'[A-Z]{4}\d{8}', r'([A-Z]+)(_[A-Z]+)\d{6}', r'[A-Z]{6}\d{9}']
                regex = re.compile(r'\b(' + '|'.join(pattern) + r')\b')
                if description is not None:
                    accessions = [m.group() for m in regex.finditer(description)]
                    #print('accessions', accessions)
    
                #find the strains
                if description is not None:
                    find_strains(cleaned_text)
                    #print('strain names', strains)
    
                #load data into pandas dataframe
                row_data = [orgname, accessions, strains, authority, doi, filtered_url]
                length = len(pub_df)
                pub_df.loc[length] = row_data
            #print('BREAK1')

   
    for element in driver.find_elements(By.CLASS_NAME, "tl-lowest-section"): #finds section headers
        description1 = element.text
        outer_html = element.get_attribute("outerHTML")
        if "Description of" in description1:          
            #print(outer_html)
            spans = soup.findAll('span', attrs = {'class' : 'tl-lowest-section'})
            for span in spans:
                if "Description of" in span.text:   
                    #print (span.text)
                    outer_div_id = span.find_parent('div').get('id')
                    #print(f"Outer div ID: {outer_div_id}, Text: {span.text}")
                    for element in driver.find_elements(By.ID, outer_div_id):
                        description = element.text
                        cleaned_text = remove_non_ascii(description)
                        combined_description.append(cleaned_text)
                        #print(cleaned_text)
                        #print(description)
                        
                    #find the organism names   
                    orgname = []
                    match = [r'(\S+\s+){2}(?=sp. nov)', r'(\S+\s+){2}(?=nom. nov.)', r'(\S+\s+){2}(?=SP. NOV.)', r'(\S+\s+){4}(?=subsp. nov.)']
                    regex = re.compile(r'\b(' + '|'.join(match) + r')\b')
                    if description is not None:
                        orgname = [m.group() for m in regex.finditer(description)]
                        #print('orgname', orgname)

                    #find the accessions
                    pattern = [r'[A-Z]{2}\d{6}', r'[A-Z]{4}\d{8}', r'([A-Z]+)(_[A-Z]+)\d{6}', r'[A-Z]{6}\d{9}']
                    regex = re.compile(r'\b(' + '|'.join(pattern) + r')\b')
                    if description is not None:
                        accessions = [m.group() for m in regex.finditer(description)]
                        #print('accessions', accessions)
    
                    #find the strains
                    strains = []
                    if description is not None:
                        find_strains(cleaned_text)
                        #print('strain names', strains)
    
                    #load data into pandas dataframe
                    row_data = [orgname, accessions, strains, authority, doi, filtered_url]
                    length = len(pub_df)
                    pub_df.loc[length] = row_data
                    #print('BREAK2')

        
#Close the browser window
    driver.quit()    

The polyamine pattern of the type strain of the type species NCTC 11411T mainly included 1,3-diaminopropane in addition to small amounts of putrescine and spermidine [6].



The G+C content determined by the Tm method was 43.9 mol% for the type strain and 45.3 for strain F74.


The G+C content determined by the Tm method was 43.9 mol% for the type strain and 45.3 for strain F74.


The G+C content determined from the whole genomic sequence (WGS) was 40.9 for the type strain.


/home/mcveigh/jupweb/lib/python3.11/site-packages/spacy/displacy/__init__.py:213: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


...

Strain W635T was positive in VogesProskauer reaction, hydrolysis of esculin, gelatin, Tween 80 and xanthine.


The strain W635T can reduce nitrate to nitrite but not to nitrogen.


The type strain, W635T (=KCTC 62429T=MCCC 1K03491T), was isolated from saline lake water in Lake Ayaqqum (3730 N 8930 E) in Xinjiang province, China.


The GenBank/EMBL/DDBJ accession number for the 16S rRNA gene sequence of strain W635T is MG722799.


The GenBank accession numbers for the whole-genome sequences of strain W635T, N. aestuarii JCM30752T and R. tibetensis CGMCC1.16230T are QVQC00000000, JBPBBA000000000 and JBPBBB000000000, respectively.
 


In [6]:
#optional write description to a file
#print(combined_description)
file = open(alldescriptions, "w")
file.writelines(combined_description)
file.close()

In [10]:
pd.set_option('max_colwidth', None)
pub_df['Strains'] = [', '.join(map(str, l)) for l in pub_df['Strains']]
#pub_df['Strains'] = pub_df['Strains'].astype(str) 
pub_df['Strains'] = pub_df['Strains'].astype(pd.StringDtype())
pub_df['Strains'] = pub_df['Strains'].str.replace(',', ', ')
pub_df = pub_df.drop_duplicates(subset='PublishedName', keep="first")
#try drop duplicate accessions here
pub_df.explode(['PublishedName']).reset_index(drop=True)

,PublishedName,Accessions,Strains,Authority,DOI,filtered_url
0,NaN,[],NCTC 11411T,Bisgaard and Christensen 2025,https://doi.org/10.1099/ijsem.0.006867,http://dx.doi.org/10.1099/ijsem.0.006867
1,Macrococcus equi,"[CP179735, CP179734, CP179731, CP179736, CP179732]","18KM445, DSM 118744, CCOS 2124, 18KM445T",Belhout et al. 2025,https://doi.org/10.1099/ijsem.0.006861,http://dx.doi.org/10.1099/ijsem.0.006861
2,Macrococcus animalis,"[CP179725, CP179727, CP179728, CP179724]","EM39E, DSM 118743, CCOS 2125, CCM 9439, JEK85",Belhout et al. 2025,https://doi.org/10.1099/ijsem.0.006861,http://dx.doi.org/10.1099/ijsem.0.006861
3,Pseudogemmobacter sonorensis,"[OP709268, JBEFZE000000000, JBEFZE010000000]","PA1-206BT, DSM 115559T, NCAIM B.02680T",Lange-Enyedi et al. 2025,https://doi.org/10.1099/ijsem.0.006859,http://dx.doi.org/10.1099/ijsem.0.006859
4,Streptomyces antarcticus,"[OP093992, JAPNLM000000000]","H27-S2T, DSM 114983T, CGMCC 4.7867T, 8.5 Mbp",Sahin et al. 2025,https://doi.org/10.1099/ijsem.0.006856,http://dx.doi.org/10.1099/ijsem.0.006856
5,Nioella halotolerans,"[MG722799, QVQC00000000, JBPBBA000000000, JBPBBB000000000]","W635T, KCTC 62429T, MCCC 1K03491T, JCM30752T, CGMCC1.16230T",Han et al. 2025,https://doi.org/10.1099/ijsem.0.006858,http://dx.doi.org/10.1099/ijsem.0.006858


In [ ]:
#pub_df['Strains'] = pub_df['Strains'].astype(pd.StringDtype())
#pub_df['Strains'] = pub_df['Strains'].str.replace(',', ', ')
#pub_df

In [11]:
pub2_df = pub_df.explode(['Accessions']).reset_index(drop=True)
#pub2_df
pub4_df = pub2_df.explode(['PublishedName']).reset_index(drop=True)
pub4_df.rename(columns={'Accessions' : 'accession'}, inplace=True)
#pub4_df = pub4_df.dropna()
#pub4_df = pub4_df.drop_duplicates(subset='accession', keep="first")
pub4_df=pub4_df[pub4_df['accession'].isnull() | ~pub4_df[pub4_df['accession'].notnull()].duplicated(subset='accession',keep='first')]
#pub4_df

In [12]:
df_unique= pub4_df.drop_duplicates(["accession"], keep="first")
#df_unique = df_unique.dropna()
#df_unique

### Create a dataframe of unique accessions and look up NCBI taxonomy information of each accession with srcchk

In [ ]:
#df_unique.dtypes

In [13]:
#df_unique['accession'] = df_unique['accession'].astype('str') 
df_unique.loc[:, 'accession'] = df_unique['accession'].astype('str') 

In [14]:
with open('acclist', 'w') as f:
    for text in df_unique['accession'].tolist():
        f.write(text + '\n')

In [15]:
os.system("/netopt/ncbi_tools64/bin/srcchk -i acclist -f taxname,taxid,strain -o acclist.taxdata")


Error: Malformatted ID "nan"


0

In [16]:
taxdata_file_name = (r'acclist.taxdata')    
srcchk_df = pd.read_csv(taxdata_file_name, sep='\t', index_col=None, low_memory=False)
srcchk_df.drop(columns=['Unnamed: 4'], inplace=True)
srcchk_df.rename(columns={'organism' : 'NCBIname'}, inplace=True)
srcchk_df['accession'] = srcchk_df['accession'].astype(str).replace('\.\d+', '', regex=True).astype(str)
srcchk_df = srcchk_df.dropna(subset=['NCBIname'])
srcchk_df 

,accession,NCBIname,taxid,strain
1,CP179735,Macrococcus equi,3395462.0,18KM445
2,CP179734,Macrococcus equi,3395462.0,18KM444
3,CP179731,Macrococcus equi,3395462.0,18KM245
4,CP179736,Macrococcus equi,3395462.0,18KM583
5,CP179732,Macrococcus equi,3395462.0,21KM1573
6,CP179725,Macrococcus animalis,3395467.0,JEK85
7,CP179727,Macrococcus animalis,3395467.0,EM39E
8,CP179728,Macrococcus animalis,3395467.0,18KM676
9,CP179724,Macrococcus animalis,3395467.0,21M1142
10,OP709268,Pseudogemmobacter sonorensis,2989681.0,PA1-206B


### Combine dataframes into one

In [17]:
combine_df=pd.merge(left=pub4_df, right=srcchk_df, left_on='accession', right_on='accession', how = 'outer')
combine_df = combine_df[['PublishedName', 'NCBIname', 'Strains', 'accession', 'strain', 'Authority', 'taxid', 'DOI', 'filtered_url' ]]
combine_df

,PublishedName,NCBIname,Strains,accession,strain,Authority,taxid,DOI,filtered_url
0,Macrococcus animalis,Macrococcus animalis,"EM39E, DSM 118743, CCOS 2125, CCM 9439, JEK85",CP179724,21M1142,Belhout et al. 2025,3395467.0,https://doi.org/10.1099/ijsem.0.006861,http://dx.doi.org/10.1099/ijsem.0.006861
1,Macrococcus animalis,Macrococcus animalis,"EM39E, DSM 118743, CCOS 2125, CCM 9439, JEK85",CP179725,JEK85,Belhout et al. 2025,3395467.0,https://doi.org/10.1099/ijsem.0.006861,http://dx.doi.org/10.1099/ijsem.0.006861
2,Macrococcus animalis,Macrococcus animalis,"EM39E, DSM 118743, CCOS 2125, CCM 9439, JEK85",CP179727,EM39E,Belhout et al. 2025,3395467.0,https://doi.org/10.1099/ijsem.0.006861,http://dx.doi.org/10.1099/ijsem.0.006861
3,Macrococcus animalis,Macrococcus animalis,"EM39E, DSM 118743, CCOS 2125, CCM 9439, JEK85",CP179728,18KM676,Belhout et al. 2025,3395467.0,https://doi.org/10.1099/ijsem.0.006861,http://dx.doi.org/10.1099/ijsem.0.006861
4,Macrococcus equi,Macrococcus equi,"18KM445, DSM 118744, CCOS 2124, 18KM445T",CP179731,18KM245,Belhout et al. 2025,3395462.0,https://doi.org/10.1099/ijsem.0.006861,http://dx.doi.org/10.1099/ijsem.0.006861
5,Macrococcus equi,Macrococcus equi,"18KM445, DSM 118744, CCOS 2124, 18KM445T",CP179732,21KM1573,Belhout et al. 2025,3395462.0,https://doi.org/10.1099/ijsem.0.006861,http://dx.doi.org/10.1099/ijsem.0.006861
6,Macrococcus equi,Macrococcus equi,"18KM445, DSM 118744, CCOS 2124, 18KM445T",CP179734,18KM444,Belhout et al. 2025,3395462.0,https://doi.org/10.1099/ijsem.0.006861,http://dx.doi.org/10.1099/ijsem.0.006861
7,Macrococcus equi,Macrococcus equi,"18KM445, DSM 118744, CCOS 2124, 18KM445T",CP179735,18KM445,Belhout et al. 2025,3395462.0,https://doi.org/10.1099/ijsem.0.006861,http://dx.doi.org/10.1099/ijsem.0.006861
8,Macrococcus equi,Macrococcus equi,"18KM445, DSM 118744, CCOS 2124, 18KM445T",CP179736,18KM583,Belhout et al. 2025,3395462.0,https://doi.org/10.1099/ijsem.0.006861,http://dx.doi.org/10.1099/ijsem.0.006861
9,Streptomyces antarcticus,Streptomyces antarcticus,"H27-S2T, DSM 114983T, CGMCC 4.7867T, 8.5 Mbp",JAPNLM000000000,H27-S2,Sahin et al. 2025,2996458.0,https://doi.org/10.1099/ijsem.0.006856,http://dx.doi.org/10.1099/ijsem.0.006856


In [18]:
def highlight_rows(row):
    ijsemvalue = row.loc['PublishedName']
    ncbivalue = row.loc['NCBIname']
    if ijsemvalue != ncbivalue:
        color = '#FFB3BA' # Red
    elif ijsemvalue == ncbivalue:
        color = '#BAFFC9' # Green
    return ['background-color: {}'.format(color) for r in row]

new_df = combine_df.style.apply(highlight_rows, axis=1, subset=['PublishedName', 'NCBIname'])
new_df

,PublishedName,NCBIname,Strains,accession,strain,Authority,taxid,DOI,filtered_url
0,Macrococcus animalis,Macrococcus animalis,"EM39E, DSM 118743, CCOS 2125, CCM 9439, JEK85",CP179724,21M1142,Belhout et al. 2025,3395467.000000,https://doi.org/10.1099/ijsem.0.006861,http://dx.doi.org/10.1099/ijsem.0.006861
1,Macrococcus animalis,Macrococcus animalis,"EM39E, DSM 118743, CCOS 2125, CCM 9439, JEK85",CP179725,JEK85,Belhout et al. 2025,3395467.000000,https://doi.org/10.1099/ijsem.0.006861,http://dx.doi.org/10.1099/ijsem.0.006861
2,Macrococcus animalis,Macrococcus animalis,"EM39E, DSM 118743, CCOS 2125, CCM 9439, JEK85",CP179727,EM39E,Belhout et al. 2025,3395467.000000,https://doi.org/10.1099/ijsem.0.006861,http://dx.doi.org/10.1099/ijsem.0.006861
3,Macrococcus animalis,Macrococcus animalis,"EM39E, DSM 118743, CCOS 2125, CCM 9439, JEK85",CP179728,18KM676,Belhout et al. 2025,3395467.000000,https://doi.org/10.1099/ijsem.0.006861,http://dx.doi.org/10.1099/ijsem.0.006861
4,Macrococcus equi,Macrococcus equi,"18KM445, DSM 118744, CCOS 2124, 18KM445T",CP179731,18KM245,Belhout et al. 2025,3395462.000000,https://doi.org/10.1099/ijsem.0.006861,http://dx.doi.org/10.1099/ijsem.0.006861
5,Macrococcus equi,Macrococcus equi,"18KM445, DSM 118744, CCOS 2124, 18KM445T",CP179732,21KM1573,Belhout et al. 2025,3395462.000000,https://doi.org/10.1099/ijsem.0.006861,http://dx.doi.org/10.1099/ijsem.0.006861
6,Macrococcus equi,Macrococcus equi,"18KM445, DSM 118744, CCOS 2124, 18KM445T",CP179734,18KM444,Belhout et al. 2025,3395462.000000,https://doi.org/10.1099/ijsem.0.006861,http://dx.doi.org/10.1099/ijsem.0.006861
7,Macrococcus equi,Macrococcus equi,"18KM445, DSM 118744, CCOS 2124, 18KM445T",CP179735,18KM445,Belhout et al. 2025,3395462.000000,https://doi.org/10.1099/ijsem.0.006861,http://dx.doi.org/10.1099/ijsem.0.006861
8,Macrococcus equi,Macrococcus equi,"18KM445, DSM 118744, CCOS 2124, 18KM445T",CP179736,18KM583,Belhout et al. 2025,3395462.000000,https://doi.org/10.1099/ijsem.0.006861,http://dx.doi.org/10.1099/ijsem.0.006861
9,Streptomyces antarcticus,Streptomyces antarcticus,"H27-S2T, DSM 114983T, CGMCC 4.7867T, 8.5 Mbp",JAPNLM000000000,H27-S2,Sahin et al. 2025,2996458.000000,https://doi.org/10.1099/ijsem.0.006856,http://dx.doi.org/10.1099/ijsem.0.006856


### write output to excel

In [41]:
new_df.to_excel(output, engine='xlsxwriter', index = False, na_rep = '') 

In [ ]:
combine_df.dtypes